# Spark Exercise #2 - Wikipedia

### Setup 
To start, first download the data for the assignment (133 MB):
[wikipedia.dat](http://alaska.epfl.ch/~dockermoocs/bigdata/wikipedia.dat) and upload it to Databricks.

### The Assignment

In this assignment, you will get to know Spark by exploring full-text Wikipedia articles.

Gauging how popular a programming language is important for companies judging whether or not they should adopt an emerging programming language. 
For that reason, industry analyst firm RedMonk has bi-annually computed a ranking of programming language popularity using a variety of data sources, 
typically from websites like GitHub and StackOverflow. 
See their [top-20 ranking for June 2016](https://redmonk.com/sogrady/2016/07/20/language-rankings-6-16/) as an example.

In this assignment, we'll use our full-text data from Wikipedia to produce a rudimentary metric of how popular a programming language is, 
in an effort to see if our Wikipedia-based rankings bear any relation to the popular Red Monk rankings.

Try to look into [PySpark Docs](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) whenever you need help

### Data Exploration
First step, we will get to know the data. Read it as a Spark RDD, try to understand it and get some insights about it.

In [3]:
rdd = sc.textFile('/FileStore/tables/wikipedia.dat')
sample = rdd.take(1)[0]
sample

In [4]:
# Play with the sample and with the RDD here to get more insights.
# For example, how many articles are in the dataset?
rdd.count()

### Read-in Wikipedia Data
From the sample we can see how each articles is composed. Each one looks like an HTML string.

Our job now is to parse this data into a PairRDD with the article's title as the key and its text as the value.

In [6]:
tags = "</title><text>"
def parse_wikipedia_article(line):
  i = line.index(tags)
  title = line[14:i]
  text = line[i+len(tags): len(line)-16]
  return (title, text)

In [7]:
rdd_wiki_articles = rdd.map(lambda l: parse_wikipedia_article(l))

## Compute a ranking of programming languages
We will use a simple metric for determining the popularity of a programming language: the number of Wikipedia articles that mention the language at least once.

### Rank languages attempt #1: Count appearances
For each language in `LANGS` list, count in how many articles it appears.

The list should be sorted in descending order. That is, according to this ranking, the pair with the highest second component (the count) should be the first element of the list.

Pay attention to roughly how long it takes to run this part! (It should take tens of seconds.)

Example of the output:
```
[('javascript', 1721), ('c#', 706), ('java', 618), ...]
```

In [9]:
LANGS = ["javascript", "java", "php", "python", "c#", "c++", "ruby", "css", "objective-c", "perl", "scala", "haskell", "matlab", "clojure", "groovy"]

In [10]:
# Write your code here
# Option 1 - For each language, filter the whole RDD
results = []
for l in LANGS:
  counter = rdd_wiki_articles.filter(lambda article: l in article[1].lower().split(' ')).count()
  results.append((l, counter))
sorted(results, key=lambda x: x[1], reverse=True)

# Option 2 - For each article, count all languages - Better performance since it doesn't read the whole RDD many times
def count_langs(line):
  lang_dict = {}
  for l in LANGS:
    if l in line.lower().split(' '):
      lang_dict[l] = 1
  return lang_dict

results2 = rdd_wiki_articles\
            .map(lambda article: count_langs(article[1]))\
            .filter(lambda d: len(d)>0)\
            .reduce(lambda d1, d2: { key: d1.get(key, 0) + d2.get(key, 0) for key in set(d1) | set(d2) })
sorted(results2.items(), key=lambda x: x[1], reverse=True)

### Rank languages attempt #2: Compute an inverted index

An inverted index is an index data structure storing a mapping from content, such as words or numbers, to a set of documents. In particular, the purpose of an inverted index is to allow fast full text searches. In our use-case, an inverted index would be useful for mapping from the names of programming languages to the collection of Wikipedia articles that mention the name at least once.

To make working with the dataset more efficient and more convenient, implement a method that computes an "inverted index" which maps programming language names to the Wikipedia articles on which they occur at least once.

Hint: You might want to use methods flatMap and groupByKey on RDD for this part.


#### Computing the ranking

Like in part 1, you should compute a list of pairs where the second component of the pair is the number of articles that mention the language (the first component of the pair is the name of the language). This time, you'll use the inverted index created above.

Again, the list should be sorted in descending order. That is, according to this ranking, the pair with the highest second component (the count) should be the first element of the list.

Hint: method mapValues on PairRDD could be useful for this part.

Can you notice a performance improvement over attempt #2? Why?

In [12]:
# Write your code here
# There are many ways of solving this:
# 1) You can create an rdd with the mapping {word -> list of articles}
# 2) Also an rdd but with the mapping {word -> amount of articles}
# Afterwards you can collect the rdd into memory and find the relevant languages with python.
# Possible improvements:
# A) To improve this, you could filter the rdds before the collect() so they contain only the languages.
# B) Use reduceByKey instead of groupByKey()

# Option 1 - Group By Key and create a list
inverted_index = rdd_wiki_articles\
  .map(lambda article: (article[0].lower(), list(set(article[1].lower().split(' ')))))\
  .flatMap(lambda art: [(word, art[0]) for word in art[1] if word != ''])\
  .groupByKey()\
  .mapValues(list)\
  .collect()
inverted_index

# Option 2 - Group By Key and calculate values's size
# inverted_index = rdd_wiki_articles\
#   .map(lambda article: (article[0].lower(), list(set(article[1].lower().split(' ')))))\
#   .flatMap(lambda art: [(word, art[0]) for word in art[1] if word != ''])\
#   .groupByKey()\
#   .mapValues(len)\
#   .collect()
# inverted_index

# Option 1 with improvement B - ReduceByKey
# inverted_index = rdd_wiki_articles\
#   .map(lambda article: (article[0].lower(), list(set(article[1].lower().split(' ')))))\
#   .flatMap(lambda art: [(word, art[0]) for word in art[1] if word != ''])\
#   .reduceByKey(lambda a,b: (a if type(a) == list else [a]) + (b if type(b) == list else [b]))\
#   .collect()
# inverted_index
#  .reduceByKey(lambda a,b: (a if type(a) == list else [a]) + (b if type(b) == list else [b]))\
results = {l:0 for l in LANGS}
for word, articles in inverted_index:
    if word in LANGS:
      results[word] += len(articles) if type(articles) == list else 1
results = [(lang, counter) for lang, counter in results.items()]
sorted(results, key=lambda x: x[1], reverse=True)